In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import io
import gc
import librosa
import seaborn as sns
import psycopg2
import sqlite3
import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async' # to fix GPU issues
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.activations import relu, tanh, linear
from tensorflow.keras.utils import Progbar, to_categorical
from tensorflow.keras import backend as K
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from scipy.signal import butter, lfilter
from joblib import Parallel, delayed #Paralleize calculation
from sqlalchemy import create_engine, Column, Integer, ARRAY, MetaData, Table, Text, TypeDecorator, LargeBinary, BLOB
from sqlalchemy.dialects.postgresql import ARRAY as PG_ARRAY
from psycopg2.extensions import register_adapter, AsIs
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import backend as K



In [2]:
#WICHTIG!!!
database_sqlite3 = False # Set True for sqlite3 and False for postgre

In [3]:
# Custom adapter function for postgre
def adapt_numpy_ndarray(numpy_array):
    return AsIs(list(numpy_array))
# Register the postgre-adapter
register_adapter(np.ndarray, adapt_numpy_ndarray)

# Function to convert the mess of an sqlite-BLOB-column
def convert_binary_to_array(binary_data):
    if binary_data is not None:
        out = io.BytesIO(binary_data)
        return np.load(out, allow_pickle=True)
    return None

# Database connection parameters and alchemy engine
dbname = 'bathunting'
user = 'python'
password = 'python_password'
host = 'localhost'
port = '5432' 

query_flavour = ''
if database_sqlite3:
    # sqllite3
    engine = create_engine('sqlite:///batcallsv14.db')
    table_name = 'batcalls'
    array_col = 'arr'
else:
    #postgres
    engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{dbname}')
    table_name = 'batcall'
    array_col = 'new_arr'
    query_flavour = '10 < ANY(new_arr) and'

def get_target_data(target, limit=0, no_target=False):
    lmt = "" if limit<=0 else f"LIMIT {limit}"
    #query = ""
    if no_target:
        query = f"SELECT {array_col} FROM {table_name} where {query_flavour} target = {target} {lmt}"
    else:
        query = f"SELECT target, {array_col} FROM {table_name} where {query_flavour} target = {target} {lmt}"
    df = pd.read_sql_query(query, engine)
    if database_sqlite3:
        df['new_arr'] = df['arr'].apply(convert_binary_to_array)
        df.drop('arr', axis=1, inplace=True)
    if no_target:
        df = pd.DataFrame(df['new_arr'].tolist())
    return df

def get_data(targets=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18], limit=0, no_target=False):
    all_df = Parallel(n_jobs=-3, prefer="threads")(delayed(get_target_data)(target, limit, no_target) for target in targets)
    df = pd.concat(all_df)
    return df

def get_targets():
    #conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host)
    #cursor = conn.cursor()
    query = f"SELECT target, bat FROM {table_name} group by target, bat order by target"
    df = pd.read_sql_query(query, engine)
    #conn.close()
    return df

def get_shape(nested_list):
    try:
        # Initialize shape list
        shape = []
        # Iterate to calculate the shape
        while isinstance(nested_list, list) or isinstance(nested_list, np.ndarray):
            shape.append(len(nested_list))
            nested_list = nested_list[0]
        return tuple(shape)
    except (TypeError, IndexError) as e:
        # In case the nested lists are not uniformly sized
        return f"Irregular shape - nested lists are not of equal size. \n ERROR: {e}"
    
# Get data to work with
def get_features_and_targets(limit=500, scaler=StandardScaler(), categorical=True):
    data = get_data(limit=limit)
    df = pd.DataFrame(data["new_arr"].tolist())
    if scaler != None:
        df = scaler.fit_transform(df)

    labels = pd.DataFrame(data["target"])
    if categorical:
        labels = to_categorical(labels, num_classes=19)
    return df, labels

def fourie_transformation(df):
    data_reshaped = []
    for _,data in df.iterrows():
        # Normalize
        data -= np.mean(data)
        data /= np.std(data)
        # Realy no idea just assuming prof did it right
        # Calculate spectrogram with FFT
        stft = np.abs(librosa.stft(np.array(data), n_fft=512, hop_length=32))
        stft = 10 * np.log10(stft)
        stft = np.nan_to_num(stft)
        # Scale between [0,1] and reduce shape if needed
        stft = (stft - np.min(stft)) / (np.max(stft) - np.min(stft))
        data_reshaped.append(stft.flatten())
    return np.array(data_reshaped)

def butter_bandpass(lowcut, highcut, fs, order=5):
  nyq = 0.5 * fs
  low = lowcut / nyq
  high = highcut / nyq
  b, a = butter(order, [low, high], btype='band')
  return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
  b, a = butter_bandpass(lowcut, highcut, fs, order=order)
  y = lfilter(b, a, data)
  return y
    
def bandpass_fourie_transformation(df):
    data_reshaped = []
    for _,data in df.iterrows():
        # Normalize
        data = data.astype(np.float32) / 32768.0
    
        # Bandpass to filter low and high frequencies
        data = butter_bandpass_filter(data, 1500, 12000, 44100, 5)
        data -= np.mean(data)
        data /= np.std(data)
        # Realy no idea just assuming prof did it right
        # Calculate spectrogram with FFT
        stft = np.abs(librosa.stft(np.array(data), n_fft=512, hop_length=32))
        stft = 10 * np.log10(stft)
        stft = np.nan_to_num(stft)
        # Scale between [0,1] and reduce shape if needed
        stft = (stft - np.min(stft)) / (np.max(stft) - np.min(stft))
        data_reshaped.append(stft.flatten())
    return np.array(data_reshaped)

def visualize_history(history, title_appendix=''):
    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'Model accuracy {title_appendix}')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f'History/accuracy_{title_appendix}.png')
    plt.close()
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'Model loss {title_appendix}')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f'History/loss_{title_appendix}.png')
    plt.close()

def generate_confusion_matrix(X_test, y_test, title_appendix):
    # Confusion Matrix
    predictions = model.predict(X_test)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = np.argmax(y_test, axis=1)  # y_test are the true labels (one-hot encoded)
    conf_matrix = confusion_matrix(true_labels, predicted_labels)
    
    # Plot the confusion matrix using Seaborn
    sns.heatmap(conf_matrix, annot=True, fmt='g')
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.savefig(f'Confusion/ConfusionMatrix_PCA_{title_appendix}')

In [4]:
df, labels = get_features_and_targets(categorical=False)
df_fourie = fourie_transformation(pd.DataFrame(df))
df_fourie_bandpass = bandpass_fourie_transformation(pd.DataFrame(df))

print(df_fourie.shape)
print(df_fourie_bandpass.shape)

pca = PCA(n_components=0.99)  # You can change the number of components
pca_fourie = PCA(n_components=0.99)
pca_fourie_bandpass = PCA(n_components=0.99)
pca.fit(df)
pca_fourie.fit(df_fourie)
pca_fourie_bandpass.fit(df_fourie_bandpass)
df_pca = pca.transform(df)
df_fourie_pca = pca_fourie.transform(df_fourie)
df_fourie_bandpass_pca = pca_fourie_bandpass.transform(df_fourie_bandpass)
#print("Explained variance ratio:", pca.explained_variance_ratio_)
#print("Components:", pca.components_)
print(f'Number of principal components (plain): {len(pca.components_)}')
print(f'Number of principal components (fourie): {len(pca_fourie.components_)}')
print(f'Number of principal components (fourie & bandpass): {len(pca_fourie_bandpass.components_)}')

(9067, 35466)
(9067, 35466)
Number of principal components (plain): 1926
Number of principal components (fourie): 5277
Number of principal components (fourie & bandpass): 5114


Die Features scheinen nach der Fourie-Trasnsformation und auch durch das Noize-Filtering schwerer durch die PCA beschrieben werden zu können als die unverarbeiteten Daten. (Es werden mehr principal components benötigt um die Varianz in den Daten zu 99% erklären zu können)

PS: Die CPU hasst das ;)

## Neuronal Network

Wie gut lassen sich die Fledermäuse klassifizieren? Zu Beginn wird mit einem sehr simplen Neuronalen Netzwerk getestet wobei die Anzahl der Neuronen variiert wird. Dies wird sowohl für die Rohdaten, die nur mit der Fourie-Transformation vorverarbeiteten Daten als auch für die zusätzlich gefilterten Daten getestet.

In [20]:
# Tests with the plain pca data
X_train, X_test, y_train, y_test = train_test_split(df_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
# Density 50 to 2000
for opti in ['RMSP', 'ADAM']:
    for i in range(50, 2001, 50):
        title_appendix = f'd{i}'
        # Build the model
        model = Sequential()
        model.add(Dense(i, activation='relu'))
        model.add(Dense(19, activation='softmax'))
    
        # Compile the model
        optimizer=RMSprop(learning_rate=0.001)
        if opti == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
        # Fit the model
        history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2)
    
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test)
        print('Density: %.i' % (i))
        print(f"Final Training Loss: {history.history['loss'][-1]}")
        print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
        print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
        print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
        visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')

        # Clear Keras session
        K.clear_session()
        del model
        gc.collect()

2024-01-17 10:14:29.717690: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f1c9c0088f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-17 10:14:29.717745: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-01-17 10:14:29.728660: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-01-17 10:14:29.863727: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-01-17 10:14:29.954648: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


57/57 [==============================] - 0s 2ms/step - loss: 8.3054 - accuracy: 0.3219
Density: 50
Final Training Loss: 3.959172772738384e-06
Final Training Accuracy: 1.0
Final Validation Loss: 7.928843021392822
Final Validation Accuracy: 0.3535492718219757
57/57 [==============================] - 0s 2ms/step - loss: 6.5931 - accuracy: 0.3611
Density: 100
Final Training Loss: 2.7532153126230696e-06
Final Training Accuracy: 1.0
Final Validation Loss: 6.4291768074035645
Final Validation Accuracy: 0.3687112331390381
57/57 [==============================] - 0s 2ms/step - loss: 6.4427 - accuracy: 0.3666
Density: 150
Final Training Loss: 2.312478500243742e-06
Final Training Accuracy: 1.0
Final Validation Loss: 6.256906032562256
Final Validation Accuracy: 0.3756030201911926
57/57 [==============================] - 0s 2ms/step - loss: 6.6150 - accuracy: 0.3671
Density: 200
Final Training Loss: 2.051109959211317e-06
Final Training Accuracy: 1.0
Final Validation Loss: 6.519431114196777
Final Val

Wie man in den zugehörigen Graphen schön sehen kann findet sehr schnell (bereits vor der 5. Epoche) Overfitting statt unabhängig von der Anzahl der Neuronen. Der Durchlauf wird daher mit early Stopping wiederholt.

In [23]:
# Tests with the plain pca data
X_train, X_test, y_train, y_test = train_test_split(df_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
# Density 50 to 2000
for opti in ['RMSP', 'ADAM']:
    for i in range(50, 2001, 50):
        title_appendix = f'd{i}'
        # Build the model
        model = Sequential()
        model.add(Dense(i, activation='relu'))
        model.add(Dense(19, activation='softmax'))
    
        # Compile the model
        optimizer=RMSprop(learning_rate=0.001)
        if opti == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
        # Fit the model
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
        history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test)
        print('Density: %.i' % (i))
        print(f"Final Training Loss: {history.history['loss'][-1]}")
        print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
        print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
        print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
        visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')

        # Clear Keras session
        K.clear_session()
        del model
        gc.collect()

Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 4.9745 - accuracy: 0.3280
Density: 50
Final Training Loss: 0.05138752609491348
Final Training Accuracy: 0.987590491771698
Final Validation Loss: 4.871613025665283
Final Validation Accuracy: 0.35148173570632935
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 5.1549 - accuracy: 0.3418
Density: 100
Final Training Loss: 0.018845876678824425
Final Training Accuracy: 0.9956911206245422
Final Validation Loss: 4.9424943923950195
Final Validation Accuracy: 0.3652653396129608
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 5.3053 - accuracy: 0.3423
Density: 150
Final Training Loss: 0.02094409614801407
Final Training Accuracy: 0.9931058287620544
Final Validation Loss: 4.9713287353515625
Final Validation Accuracy: 0.3728463053703308
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 5.2438 - accuracy: 0.3523

In [22]:
# Tests with fourie transformation only
X_train, X_test, y_train, y_test = train_test_split(df_fourie_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
# Density 50 to 2000
for opti in ['RMSP', 'ADAM']:
    for i in range(50, 2001, 50):
        title_appendix = f'd{i}'
        # Build the model
        model = Sequential()
        model.add(Dense(i, activation='relu'))
        model.add(Dense(19, activation='softmax'))
    
        # Compile the model
        optimizer=RMSprop(learning_rate=0.001)
        if opti == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
        # Fit the model
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
        history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test)
        print('Density: %.i' % (i))
        print(f"Final Training Loss: {history.history['loss'][-1]}")
        print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
        print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
        print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
        visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')

        # Clear Keras session
        K.clear_session()
        del model
        gc.collect()

Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.8674 - accuracy: 0.7630
Density: 50
Final Training Loss: 0.00015400766278617084
Final Training Accuracy: 1.0
Final Validation Loss: 0.8316102027893066
Final Validation Accuracy: 0.7849758863449097
Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.8124 - accuracy: 0.7817
Density: 100
Final Training Loss: 0.00011275355063844472
Final Training Accuracy: 1.0
Final Validation Loss: 0.7538713216781616
Final Validation Accuracy: 0.7973811030387878
Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.7961 - accuracy: 0.7883
Density: 150
Final Training Loss: 9.916954149957746e-05
Final Training Accuracy: 1.0
Final Validation Loss: 0.7595594525337219
Final Validation Accuracy: 0.7932460308074951
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.7899 - accuracy: 0.7922
Density: 200
Final Training Loss: 0.

In [24]:
# Tests with butter bandpass and fourie transformation
X_train, X_test, y_train, y_test = train_test_split(df_fourie_bandpass_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
# Density 50 to 2000
for opti in ['RMSP', 'ADAM']:
    for i in range(50, 2001, 50):
        title_appendix = f'd{i}'
        # Build the model
        model = Sequential()
        model.add(Dense(i, activation='relu'))
        model.add(Dense(19, activation='softmax'))
    
        # Compile the model
        optimizer=RMSprop(learning_rate=0.001)
        if opti == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
        # Fit the model
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
        history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test)
        print('Density: %.i' % (i))
        print(f"Final Training Loss: {history.history['loss'][-1]}")
        print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
        print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
        print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
        visualize_history(history, f'PCA_fourie_bandpass_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
        
        # Clear Keras session
        K.clear_session()
        del model
        gc.collect()

Epoch 8: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.8616 - accuracy: 0.7806
Density: 50
Final Training Loss: 0.0002594952820800245
Final Training Accuracy: 1.0
Final Validation Loss: 0.7704319357872009
Final Validation Accuracy: 0.7856650352478027
Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.8209 - accuracy: 0.7889
Density: 100
Final Training Loss: 0.00023736979346722364
Final Training Accuracy: 1.0
Final Validation Loss: 0.7373093366622925
Final Validation Accuracy: 0.7904893159866333
Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.8153 - accuracy: 0.7889
Density: 150
Final Training Loss: 0.00020279006275814027
Final Training Accuracy: 1.0
Final Validation Loss: 0.7196061611175537
Final Validation Accuracy: 0.7898001670837402
Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.8330 - accuracy: 0.7905
Density: 200
Final Training Loss: 0.

Während die mit der PCA transformierten Rohdaten mit einem einfachen Neuronalen Netz nur eine Genauigkeit von weniger als 40% erreicht, schaffen es die Neuronalen Netze welche mit den Daten lernen die aus vorverarbeiteten Daten durch die PCA erzugt werden auf um die 80%. Dies liegt mit Sicherheit auch daran, dass die Anzahl der Principal Components (und damit der Daten allgemein) mehr als doppelt so groß ist. 

Da die Model-Performance für die nur mit der Fourie-Transformation vorverarbeiteten Daten und die Daten welche zusätzlich noch gefiltert wurden nahezu identisch ist werden im Folgenden nur noch die Fourie-Transformierten Werte verwendet. Nun wird überprüft welchen Einfluss die Anzahl der Layer auf die Modelperformance hat. 

In [22]:
# Tests with fourie transformation only
# Add more Layer; starting with 1024 neurons; decrease the number of neurons in every following layer;
X_train, X_test, y_train, y_test = train_test_split(df_fourie_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
for opti in ['RMSP', 'ADAM']:
    for i in range(6):
        title_appendix = f'layer#{11-i}_optimizer={opti}'
        # Build the model
        model = Sequential()
        model.add(Dense(pow(2,10), activation='relu'))
        for x in range(i):
            model.add(Dense(pow(2,10-(x+1)), activation='relu'))
        model.add(Dense(19, activation='softmax'))
        optimizer=RMSprop(learning_rate=0.001)
        if opti == 'ADAM':
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
        # Fit the model
        early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
        history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2, callbacks=[early_stopping])
    
        # Evaluate the model
        loss, accuracy = model.evaluate(X_test, y_test)
        print('Layer: %.i; Optimizer: %s' % (i+1, opti))
        print(f"Final Training Loss: {history.history['loss'][-1]}")
        print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
        print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
        print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
        visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
    
        # Clear Keras session
        K.clear_session()
        del model
        gc.collect()

Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.7879 - accuracy: 0.8065
Layer: 1; Optimizer: RMSP
Final Training Loss: 6.911968375789002e-05
Final Training Accuracy: 1.0
Final Validation Loss: 0.7274727821350098
Final Validation Accuracy: 0.8035837411880493
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 1.5725 - accuracy: 0.7767
Layer: 2; Optimizer: RMSP
Final Training Loss: 7.593476038891822e-05
Final Training Accuracy: 1.0
Final Validation Loss: 1.4059661626815796
Final Validation Accuracy: 0.7884217500686646
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 2.4062 - accuracy: 0.7304
Layer: 3; Optimizer: RMSP
Final Training Loss: 0.02097475156188011
Final Training Accuracy: 0.9944846630096436
Final Validation Loss: 2.212620496749878
Final Validation Accuracy: 0.7574086785316467
Epoch 6: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 2.3575 - acc

Über alle verwendeten Methoden hinweg sieht man, dass das Early stopping sehr früh greift (nach 6 bzw. 7 Epochen) bedenkt man, dass die patience auf 5 gesetzt wurde heißt das, dass die Modelle bereits am Anfang ins Overfitting übergehen. Da dieses Verhalten unter anderem auf eine zu große Lernrate hinweisen kann wird im Folgenden mit geringeren Lernraten experimentiert.  

In [24]:
# Tests with fourie transformation only
X_train, X_test, y_train, y_test = train_test_split(df_fourie_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
for opti in ['RMSP', 'ADAM']:
    for i in range(3):
        for lr in [0.0005,0.0001,0.00005]:
            title_appendix = f'layer{11-i}_optimizer={opti}_lr={lr}'
            # Build the model
            model = Sequential()
            model.add(Dense(pow(2,10), activation='relu'))
            for x in range(i):
                model.add(Dense(pow(2,10-(x+1)), activation='relu'))
            model.add(Dense(19, activation='softmax'))
            optimizer=RMSprop(learning_rate=lr)
            if opti == 'ADAM':
                optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
            model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
        
            # Fit the model
            early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2, callbacks=[early_stopping])
        
            # Evaluate the model
            loss, accuracy = model.evaluate(X_test, y_test)
            print('Layer: %.i; Optimizer: %s; (Starting-)Learning-Rate: %.5f' % (i+1, opti, lr))
            print(f"Final Training Loss: {history.history['loss'][-1]}")
            print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
            print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
            print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
            visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
        
            # Clear Keras session
            K.clear_session()
            del model
            gc.collect()

Epoch 7: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.7216 - accuracy: 0.7988
Layer: 1; Optimizer: RMSP; (Starting-)Learning-Rate: 0.00050
Final Training Loss: 0.00019960582721978426
Final Training Accuracy: 1.0
Final Validation Loss: 0.6797791719436646
Final Validation Accuracy: 0.8028945326805115
Epoch 12: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6519 - accuracy: 0.7938
Layer: 1; Optimizer: RMSP; (Starting-)Learning-Rate: 0.00010
Final Training Loss: 0.001491366303525865
Final Training Accuracy: 1.0
Final Validation Loss: 0.6163101196289062
Final Validation Accuracy: 0.8008270263671875
Epoch 19: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6424 - accuracy: 0.7955
Layer: 1; Optimizer: RMSP; (Starting-)Learning-Rate: 0.00005
Final Training Loss: 0.0024814556818455458
Final Training Accuracy: 1.0
Final Validation Loss: 0.5922610759735107
Final Validation Accuracy: 0.8056512475013733
Ep

Da der loss immer noch sehr schnell nach oben geht wird nun zusätzlich noch Dropoutregularisierung eingebaut. Es wird im folgenden die niedrigste gestestete Lernrate für beide Optimierer verwendet (0.00005).

In [28]:
# Tests with fourie transformation only
X_train, X_test, y_train, y_test = train_test_split(df_fourie_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
for opti in ['RMSP', 'ADAM']:
    for i in range(3):
        for dor in [0.2, 0.35, 0.5]:
            title_appendix = f'layer{11-i}_optimizer={opti}_dropout={dor}'
            # Build the model
            model = Sequential()
            model.add(Dense(pow(2,10), activation='relu'))
            model.add(Dropout(dor))
            for x in range(i):
                model.add(Dense(pow(2,10-(x+1)), activation='relu'))
                model.add(Dropout(dor))
            model.add(Dense(19, activation='softmax'))
            optimizer=RMSprop(learning_rate=0.00005)
            if opti == 'ADAM':
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
            model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
        
            # Fit the model
            early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
            history = model.fit(X_train, y_train, epochs=50, batch_size=10, verbose=0, validation_split=0.2, callbacks=[early_stopping])
        
            # Evaluate the model
            loss, accuracy = model.evaluate(X_test, y_test)
            print('Layer: %.i; Optimizer: %s; Dropout-Rate: %.2f' % (i+1, opti, dor))
            print(f"Final Training Loss: {history.history['loss'][-1]}")
            print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
            print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
            print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
            visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
        
            # Clear Keras session
            K.clear_session()
            del model
            gc.collect()

Epoch 21: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6610 - accuracy: 0.7911
Layer: 1; Optimizer: RMSP; Dropout-Rate: 0.20
Final Training Loss: 0.002768370322883129
Final Training Accuracy: 1.0
Final Validation Loss: 0.6174325346946716
Final Validation Accuracy: 0.7966919541358948
Epoch 20: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6484 - accuracy: 0.7933
Layer: 1; Optimizer: RMSP; Dropout-Rate: 0.35
Final Training Loss: 0.004757148679345846
Final Training Accuracy: 1.0
Final Validation Loss: 0.6148341298103333
Final Validation Accuracy: 0.7904893159866333
Epoch 22: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6625 - accuracy: 0.7933
Layer: 1; Optimizer: RMSP; Dropout-Rate: 0.50
Final Training Loss: 0.00601459015160799
Final Training Accuracy: 1.0
Final Validation Loss: 0.6206346750259399
Final Validation Accuracy: 0.793935239315033
Epoch 13: early stopping
57/57 [===================

Mit der reduzierten Lernrate und den zusätzlichen Dropout-Layern lässt sich zwar das Overfitting effektiv verhindern, die Modellperformance zeigt leider keine Verbesserung. Als letzter Versuch werden noch verschiedene batchsizes ausprobiert.

In [29]:
# Tests with fourie transformation only
X_train, X_test, y_train, y_test = train_test_split(df_fourie_pca, to_categorical(labels, num_classes=19), test_size=0.2, random_state=42)
for opti in ['RMSP', 'ADAM']:
    for i in range(3):
        for bs in [25,50,100]:
            title_appendix = f'layer{11-i}_optimizer={opti}_batchsize={bs}'
            # Build the model
            model = Sequential()
            model.add(Dense(pow(2,10), activation='relu'))
            model.add(Dropout(dor))
            for x in range(i):
                model.add(Dense(pow(2,10-(x+1)), activation='relu'))
                model.add(Dropout(0.2))
            model.add(Dense(19, activation='softmax'))
            optimizer=RMSprop(learning_rate=0.00005)
            if opti == 'ADAM':
                optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)
            model.compile(loss='categorical_crossentropy',
                          optimizer=optimizer,
                          metrics=['accuracy'])
        
            # Fit the model
            early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')
            history = model.fit(X_train, y_train, epochs=50, batch_size=bs, verbose=0, validation_split=0.2, callbacks=[early_stopping])
        
            # Evaluate the model
            loss, accuracy = model.evaluate(X_test, y_test)
            print('Layer: %.i; Optimizer: %s; Batchsize: %.i' % (i+1, opti, bs))
            print(f"Final Training Loss: {history.history['loss'][-1]}")
            print(f"Final Training Accuracy: {history.history['accuracy'][-1]}")
            print(f"Final Validation Loss: {history.history['val_loss'][-1]}")
            print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]}")
            visualize_history(history, f'PCA_fourie_only_{title_appendix}_dens={i}_finacc={history.history["val_accuracy"][-1]:.2f}')
        
            # Clear Keras session
            K.clear_session()
            del model
            gc.collect()

Epoch 36: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6405 - accuracy: 0.7955
Layer: 1; Optimizer: RMSP; Batchsize: 25
Final Training Loss: 0.007414570078253746
Final Training Accuracy: 1.0
Final Validation Loss: 0.6041816473007202
Final Validation Accuracy: 0.7980703115463257
57/57 [==============================] - 0s 2ms/step - loss: 0.6255 - accuracy: 0.7971
Layer: 1; Optimizer: RMSP; Batchsize: 50
Final Training Loss: 0.011497807689011097
Final Training Accuracy: 1.0
Final Validation Loss: 0.5832279920578003
Final Validation Accuracy: 0.8035837411880493
57/57 [==============================] - 0s 2ms/step - loss: 0.6508 - accuracy: 0.7856
Layer: 1; Optimizer: RMSP; Batchsize: 100
Final Training Loss: 0.052916787564754486
Final Training Accuracy: 1.0
Final Validation Loss: 0.6140007376670837
Final Validation Accuracy: 0.7911785244941711
Epoch 22: early stopping
57/57 [==============================] - 0s 2ms/step - loss: 0.6995 - accuracy: 0.7916
L

Eine Genauigkeit von 80% scheint bei dieser Vorgehensweise nicht großartig übertroffen werden zu können. (Convolutional Neuronal Networks werden in einem anderen Notebook behandelt).